In [27]:
import time
import tensorflow as tf
import numpy as np
import os


# get paths to tf records
tf_dataset_directory = "/Users/andy/Projects/biology/research/komorebi/data/attention_validation_tf_dataset"
tf_record_paths = [os.path.join(tf_dataset_directory, tf_record) for tf_record in os.listdir(tf_dataset_directory)]

num_epochs = 2
num_samples = len(tf_record_paths)
buffer_size = 2000
batch_size = 100

In [18]:
SEQUENCE_SHAPE = (1000, 4)
ANNOTATION_SHAPE = (75, 320)

def parse_example(tf_example):
    """Parse tensorflow example"""
    
    features_map = {
        'sequence_raw': tf.FixedLenFeature([], tf.string),
        'label_raw': tf.FixedLenFeature([], tf.string),
        'annotation_raw': tf.FixedLenFeature([], tf.string)}
    
    parsed_example = tf.parse_single_example(tf_example, features_map)
    
    sequence_raw = tf.decode_raw(parsed_example['sequence_raw'], tf.uint8)
    annotation_raw = tf.decode_raw(parsed_example['annotation_raw'], tf.float32)
    
    sequence = tf.reshape(sequence_raw, SEQUENCE_SHAPE)
    label = tf.decode_raw(parsed_example['label_raw'], tf.uint8)
    annotation = tf.reshape(annotation_raw, ANNOTATION_SHAPE)
    
    return {'sequence': sequence, 'label': label, 'annotation': annotation}

In [19]:
# specify dataset initialization
filenames_op = tf.placeholder(tf.string, shape=[None])
tf_dataset = tf.data.TFRecordDataset(filenames_op)
tf_dataset = tf_dataset.prefetch(buffer_size)
tf_dataset = tf_dataset.map(parse_example, num_parallel_calls=6)
tf_dataset = tf_dataset.batch(batch_size)

iterator = tf_dataset.make_initializable_iterator()

In [28]:
# run iteration
ds_init_times = []
epoch_times = []

with tf.Session() as sess:

    # epoch loop
    for epoch in range(num_epochs):
        print "Starting epoch {}".format(epoch)
        try:
            ds_init_ts = time.time()
            
            np.random.shuffle(tf_record_paths)
            sess.run(iterator.initializer, {filenames_op: tf_record_paths})
            
            ds_init_te = time.time()
            ds_init_times.append(ds_init_te - ds_init_ts)

            # iteration loop
            epoch_ts = time.time()
            for i in xrange(num_samples):
                feed_dict = sess.run(iterator.get_next())
            
        except tf.errors.OutOfRangeError:
            epoch_te = time.time()
            epoch_times.append(epoch_te - epoch_ts)
            continue
        
        

Starting epoch 0
Starting epoch 1


In [30]:
ds_init_times

[0.07898998260498047, 0.005075216293334961]